In [1]:
from alibi.explainers.counterfactual import Counterfactual 
import pandas as pd
import tensorflow as tf
import json
tf.compat.v1.disable_eager_execution()

c:\ProgramData\Anaconda3\envs\cf\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = tf.keras.models.load_model('../models/adult_NN')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                5824      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
 dropout_2 (Dropout)         (None, 16)                0         
                                                                 
 dense_3 (Dense)             (None, 2)                 3

In [3]:
df = pd.read_csv('../data/adult_prep_test.csv')


with open('../data/adult_constraints.json', 'r') as f:
    constraints = json.load(f)

continuous = constraints['continuous']
categorical = constraints['categorical']
immutable = constraints['immutable']
columns_order = constraints['features_order']

x_train = df[columns_order].to_numpy()
y_train = pd.get_dummies(df['income']).to_numpy()
x_train

array([[1.        , 0.53333333, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.89041096, 0.53333333, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.67123288, 0.6       , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.31506849, 0.53333333, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.56164384, 0.53333333, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.06849315, 0.53333333, 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [4]:
shape = (1,) + x_train.shape[1:]
cf = Counterfactual(model, shape, distance_fn='l1', target_proba=1.0,
                    target_class='other', max_iter=1000, early_stop=50, lam_init=1e-1,
                    max_lam_steps=10, tol=0.05, learning_rate_init=0.1,
                    feature_range=(-1e10, 1e10), eps=0.01, init='identity',
                    decay=True, write_dir=None, debug=False)

c:\ProgramData\Anaconda3\envs\cf\lib\site-packages\keras\engine\training_v1.py:2357: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


In [5]:
X = x_train[0]
Y = y_train[0]
X = X.reshape((1, -1))
X.shape

(1, 90)

In [6]:
# Get Explanation
explanation = cf.explain(X)

In [7]:
explanation.data['cf']

{'X': array([[ 1.0047746e+00,  5.2436715e-01,  9.3911238e-02,  1.0120199e+00,
          4.1177741e-01,  1.5639048e-02, -3.1082109e-03,  1.3877047e-02,
         -1.3947953e-03,  1.0727939e-02, -4.3092063e-03,  9.7391188e-01,
          1.0184576e-02,  1.2461941e-03,  9.5135975e-04, -1.2238431e-03,
          4.0630698e-03, -1.7212745e-04, -6.1460789e-03,  3.1360574e-03,
          8.4790420e-03, -7.5607998e-03,  5.1693921e-03,  1.3723255e-03,
          1.8720578e-02,  1.5775738e-03, -8.8944100e-05,  6.6890977e-03,
         -6.3084499e-03,  1.0160287e-02,  5.3789346e-03,  1.4091088e-02,
          2.3128814e-03, -5.2549373e-03, -6.1742784e-03,  5.1474338e-03,
         -4.1478872e-04, -1.5279557e-02, -1.5200231e-02, -7.6019485e-04,
          1.4321678e-03, -5.2319369e-03,  1.5877543e-02,  1.1952227e-02,
         -2.8428279e-03, -5.9494837e-03,  8.4258607e-03, -8.9137992e-03,
          1.0261970e-02, -1.7468031e-02,  7.0273038e-03,  9.9328226e-01,
         -3.0106828e-03, -7.7393595e-03,  9.87

In [8]:
ctf = explanation.data['cf']['X']
print('X:', Y)
print('cf:', model.predict(ctf))


X: [1 0]
cf: [[0.02960015 0.97039986]]


Test sklearn

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from joblib import dump, load

In [10]:
clf = load('../models/adult_RF.joblib')

c:\ProgramData\Anaconda3\envs\cf\lib\site-packages\sklearn\base.py:288: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.23.2 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\ProgramData\Anaconda3\envs\cf\lib\site-packages\sklearn\base.py:288: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.23.2 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [11]:
clf.predict(X)

array([[1, 0]], dtype=uint8)

In [12]:
clf.predict(ctf)

array([[0, 1]], dtype=uint8)

In [19]:
predict_fn = lambda x: clf.predict(x)
cf = Counterfactual(predict_fn, shape, distance_fn='l1', target_proba=1.0,
                    target_class='other', max_iter=1000, early_stop=50, lam_init=1e-3,
                    max_lam_steps=15, tol=0.4, learning_rate_init=0.1,
                    feature_range=(0, 1), eps=0.1, init='identity',
                    decay=True, write_dir=None, debug=False)

In [20]:
# Get Explanation
explanation = cf.explain(X)

In [21]:
ctf = explanation.data['cf']['X']
print('X:', Y)
print('cf:', model.predict(ctf))

X: [1 0]
cf: [[0.06001339 0.93998665]]


CF PROTO

In [42]:
from alibi.explainers import CounterfactualProto

predict_fnct = lambda x: model.predict(x)

cf = CounterfactualProto(predict_fnct, shape, use_kdtree=True, theta=10., feature_range=(0.0, 1.0))
cf.fit(x_train, trustscore_kwargs=None)

No encoder specified. Using k-d trees to represent class prototypes.


CounterfactualProto(meta={
  'name': 'CounterfactualProto',
  'type': ['blackbox', 'tensorflow', 'keras'],
  'explanations': ['local'],
  'params': {
              'kappa': 0.0,
              'beta': 0.1,
              'gamma': 0.0,
              'theta': 10.0,
              'cat_vars': None,
              'ohe': False,
              'use_kdtree': True,
              'learning_rate_init': 0.01,
              'max_iterations': 1000,
              'c_init': 10.0,
              'c_steps': 10,
              'eps': (0.001, 0.001),
              'clip': (-1000.0, 1000.0),
              'update_num_grad': 1,
              'write_dir': None,
              'feature_range': (0.0, 1.0),
              'shape': (1, 90),
              'is_model': False,
              'is_ae': False,
              'is_enc': False,
              'enc_or_kdtree': True,
              'is_cat': False,
              'trustscore_kwargs': None,
              'd_type': 'abdm',
              'w': None,
              'disc_per

In [39]:
explanation = cf.explain(X, k=2)

In [40]:
explanation['data']['all']

C:\Users\ignacy\AppData\Local\Temp\ipykernel_6816\245771069.py:1: DeprecationWarning: The Explanation object is not a dictionary anymore and accessing elements should be done via attribute access. Accessing via item will stop working in a future version.
  explanation['data']['all']


{0: [array([[0.99064535, 0.5807336 , 0.4860936 , 0.8645695 , 0.39795917,
          0.        , 0.        , 0.        , 0.        , 0.        ,
          0.        , 1.        , 0.        , 0.        , 0.        ,
          0.        , 0.        , 0.        , 0.        , 0.        ,
          0.        , 0.        , 0.        , 0.        , 0.        ,
          0.        , 0.        , 0.        , 0.        , 0.        ,
          0.        , 0.        , 0.        , 0.        , 0.        ,
          0.        , 0.        , 0.        , 0.        , 0.        ,
          0.        , 0.        , 0.        , 0.        , 0.        ,
          0.        , 0.        , 0.        , 0.        , 0.        ,
          0.        , 1.        , 0.        , 0.        , 1.        ,
          0.        , 0.        , 0.        , 0.        , 0.        ,
          0.        , 0.        , 0.        , 0.        , 0.        ,
          0.        , 0.        , 0.        , 0.        , 0.        ,
          0.     